In [1]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tqdm.notebook import tqdm
from pprint import pprint
from statsmodels.stats.weightstats import ztest

In [3]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import bs4
import re
import math
import ast

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
# для красивых графиков

%config InlineBackend.figure_format = 'retina'

sns.set(style='darkgrid', palette='deep')

plt.rcParams['font.size'] = 16
plt.rcParams['savefig.format'] = 'pdf'

In [6]:
df = pd.read_csv("eda_data.csv")
df

,artist,country,listeners_lastfm,scrobbles_lastfm,likes,tracks,albums,genres,years_active
0,Coldplay,United Kingdom,5381567.0,360111850.0,1305445,186,60,['rock'],26
1,Radiohead,United Kingdom,4732528.0,499548797.0,387180,200,42,['indie'],38
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0,2155459,266,32,['rock'],41
3,Rihanna,United States,4558193.0,199248986.0,1700983,268,57,"['pop', 'dance', 'rnb']",20
4,Eminem,United States,4517997.0,199507511.0,5278564,396,37,['foreignrap'],35
...,...,...,...,...,...,...,...,...,...
3834,Lyn Collins,United States,189533.0,845689.0,109,43,4,"['rnb', 'electronics']",43
3835,Saigon,United States,189479.0,1939393.0,82,298,42,"['rap', 'pop', 'african']",23
3836,Kwabs,United Kingdom,189460.0,1655444.0,1207967,28,8,['soul'],12
3837,Diablo Swing Orchestra,Sweden,189429.0,8016066.0,15532,60,5,['progmetal'],20


# Шаг 5. Создание новых признаков

Для начала в соответствии с графиками в EDA, добавим прологарифмированные переменные, чтобы улучшить их вид.

In [7]:
df['log_listeners'] = np.log(df.listeners_lastfm)
df['log_scrobbles'] = np.log(df.scrobbles_lastfm)
df['log_likes'] = np.log(df.likes)
df['log_tracks'] = np.log(df.tracks)

Изначально мы хотели предсказывать кол-во лайков, но давайте предсказывть лучше логарифм, потому что нам было бы важнее для задачи угадывать с порядком числа, то есть по сути: является группа более любимой для слушателей, или менее.

Глобально мы хотим предсказывать переменную связанную с лайками, поэтому в создании признаков больше ее мы трогать не будем, чтобы не допустить утечки таргета.

Можно завести переменные, означающие отношение одной колонки к другой. Например это может быть отношение кол-во слушателей, кол-во скробблов, треков и альбомов к годам активности группы. А также отношение их логарифмов к годам. Это может быть важно, потому что группа может быть очень продуктивной в течение своего небольшого времени активности и написать мало альбомов, а может написать альбомов немногим большее кол-во, но за промежуток времени в 2 раза больший. Возможно первая группа будет более популярна, потому что музыка это не только про вдохновение, но и хороший менеджмент и энергию исполнителей.

In [8]:
df['listeners_per_years'] = df.listeners_lastfm / df.years_active
df['scrobbles_per_years'] = df.scrobbles_lastfm / df.years_active
df['tracks_per_years'] = df.tracks / df.years_active
df['albums_per_years'] = df.albums / df.years_active

df['log_listeners_per_years'] = df.log_listeners / df.years_active
df['log_scrobbles_per_years'] = df.log_scrobbles / df.years_active
df['log_tracks_per_years'] = df.log_tracks / df.years_active

Можно посмотреть на то, сколько треков содержится в среднем в одном альбоме.

In [9]:
df['tracks_per_album'] = df.tracks / df.albums

Кроме того, при парсинге данных мы не включили в данные год начала группы и год завершения выступлений. Давайте добавим их.

In [10]:
df_tags = pd.read_csv("big_lst_5000.csv", index_col=0)
df_tags

,artist_mb,parsed_data
0,Coldplay,"<td class=""infobox-data"">1997–present</td>"
1,Radiohead,"<td class=""infobox-data"">1985–present</td>"
2,Red Hot Chili Peppers,"<td class=""infobox-data"">1982<span style=""disp..."
3,Rihanna,"<td class=""infobox-data"">2003–present</td>"
4,Eminem,"<td class=""infobox-data"">1988–present<sup clas..."
...,...,...
4857,Kwabs,"<td class=""infobox-data"">2011–present</td>"
4858,Astral Projection,"<td class=""infobox-data"">1993–present</td>"
4859,Diablo Swing Orchestra,"<td class=""infobox-data"">2003–present</td>"
4860,Despised Icon,"<td class=""infobox-data"">2002–2010, 2014–prese..."


Воспользуемся кодом из предыдущих тетрадок.

In [11]:
def get_intervals(raw_intervals):
    intervals = []
    # split by '-'
    left_rights = re.findall(r'\S+–\S+', raw_intervals)
    for left_right in left_rights:
        parts = left_right.split('–')
        clean_parts = tuple(re.findall(r'(?:\d{4}|present)', part)[0] for part in parts)
        clean_parts_without_present = tuple(2023 if part == 'present' else int(part) for part in clean_parts)
        intervals.append(clean_parts_without_present)
    return intervals

def parse_years_text(s):
    # check hiaustes
    main_and_hiatuses = s.split('hiatus')
    main = main_and_hiatuses[0]
    hiastuses = ''
    if len(main_and_hiatuses) > 1:
        hiastuses = main_and_hiatuses[1]

    main_intervals = get_intervals(main)
    hiastuses_intervals = get_intervals(hiastuses)
        
    return main_intervals, hiastuses_intervals

def get_first_year(tag):
    try:
        years = bs4.BeautifulSoup(tag)
        intervals, _ = parse_years_text(years.text)
        return intervals[0][0]
    except:
        return None
    
def get_last_year(tag):
    try:
        years = bs4.BeautifulSoup(tag)
        intervals, _ = parse_years_text(years.text)
        return intervals[-1][1]
    except:
        return None



In [12]:
# df['first_year'] = 
# for _, row in df:
#     tag = df_tags[df_tags == row.artist].tag
#     dfget_last_year(x)).isna().sum()

### Шаг 7: Машинное обучение

In [13]:
# удаляем колонку с артистами, потому что это никак не поможет модели
df = df.drop('artist', axis=1)

In [14]:
# удаляем колонку с жанрами
df = df.drop('genres', axis=1)

In [15]:
# One Hot Encoding для country
df = pd.get_dummies(df, columns=['country'])
df.shape

(3839, 45)

In [15]:
# mlb = MultiLabelBinarizer()
# data_ohe = data_ohe.join(pd.DataFrame(mlb.fit_transform(data_ohe.pop('genres')),
#                           columns=mlb.classes_,
#                           index=data_ohe.index))
# data_ohe.shape

(3839, 76)

In [19]:
X = df.drop(['likes', 'log_likes'], axis=1)
y = df['log_likes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

3.2067663282706422

Это модель-бейзлайн. Предсказываем mse.